In [1]:
%cd ..

/home/jupyter-vhirtham/weldx/weldx


In [2]:
%matplotlib widget

In [3]:
# necesessary packages
import matplotlib.pyplot as plt
import weldx.transformations as tf
import weldx.visualization as vs
import xarray as xr
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

# Introduction

This tutorial is about the `CoordinateSystemManager` class of the `weldx.transformations` package. It builds upon the tutorial about coordinate systems (tutorials/transformations_01_coordinate_systems.ipynb).

The purpose of the `CoordinateSystemManager` is to define and manage the relationships of different coordinate systems and their associated data in a tree-like data structure. It provides methods to transform a `LocalCoordinateSystem` or its data to an arbitrary other coordinate system.

# Construct and add coordinate systems

The constructor of `CoordinateSystemManager` requires the name of the root coordinate system as parameter. 

In [4]:
csm = tf.CoordinateSystemManager("root")

> **Hint:** The name of a coordinate systems can be any hashable type even though it is advised to use strings.

Further coordinate systems can be added with the `add_coordinate_system` member function of the `CoordinateSystemManager`. It expects 3 parameters. The first parameter is the name of the new coordinate system. The second parameter is the name of an already existing coordinate system in which is the reference system of the new one. The third parameter is a `weldx.transformations.LocalCoordinateSystem` that describes the postion and orientation of the new coordinate system in its reference system. 

In [5]:
# create some coordinate systems
lcs_flange_in_root = tf.LocalCoordinateSystem.construct_from_euler(
    sequence="x", angles=20, degrees=True, coordinates=[-1, -1, 2]
)
lcs_torch_in_flange = tf.LocalCoordinateSystem(coordinates=[0, 0, -1])
lcs_specimen_in_root = tf.LocalCoordinateSystem(coordinates=[0, 1, -2])

# add coordinate systems to CSM
csm.add_coordinate_system("flange", "root", lcs_flange_in_root)
csm.add_coordinate_system("torch", "flange", lcs_torch_in_flange)
csm.add_coordinate_system("specimen", "root", lcs_specimen_in_root)

# Transform coordinate systems to another reference system

After all coordinate systems are added to the `CoordinateSystemManager`, getting a coordinate system in relation to any other reference system in form of a `LocalCoordinateSystem` class is quite easy. Simply call `get_local_coordinate_system`. The name of the system that should be transformed is passed to the function as first parameter. The second parameter is the name of the target reference system.

In [6]:
lcs_torch_in_root = csm.get_local_coordinate_system("torch", "root")

lcs_root_in_torch = csm.get_local_coordinate_system("root", "torch")
lcs_flange_in_torch = csm.get_local_coordinate_system("flange", "torch")
lcs_specimen_in_torch = csm.get_local_coordinate_system("specimen", "torch")

The following plots show the different systems in the root coordinate system and in the torch coordinate system:

In [7]:
lcs_root_in_root = tf.LocalCoordinateSystem()
lcs_torch_in_torch = tf.LocalCoordinateSystem()

# plot coordinate systems
fig_01 = plt.figure()
fig_01.set_size_inches(16, 8)
gs_01 = fig_01.add_gridspec(1, 2)
ax_0_01 = fig_01.add_subplot(gs_01[0, 0], projection="3d")
ax_1_01 = fig_01.add_subplot(gs_01[0, 1], projection="3d")

# first plot
vs.plot_coordinate_system(lcs_root_in_root, ax_0_01, color="r", label="root")
vs.plot_coordinate_system(lcs_flange_in_root, ax_0_01, color="g", label="flange")
vs.plot_coordinate_system(lcs_specimen_in_root, ax_0_01, color="y", label="specimen")
vs.plot_coordinate_system(lcs_torch_in_root, ax_0_01, color="b", label="torch")
ax_0_01.set_xlim([-3, 3])
ax_0_01.set_ylim([-3, 3])
ax_0_01.set_zlim([-3, 3])
ax_0_01.set_title("root system")
ax_0_01.legend(loc="lower left")  #


# second plot
vs.plot_coordinate_system(lcs_root_in_torch, ax_1_01, color="r", label="root")
vs.plot_coordinate_system(lcs_flange_in_torch, ax_1_01, color="g", label="flange")
vs.plot_coordinate_system(lcs_specimen_in_torch, ax_1_01, color="y", label="specimen")
vs.plot_coordinate_system(lcs_torch_in_torch, ax_1_01, color="b", label="torch")
ax_1_01.set_xlim([-3, 3])
ax_1_01.set_ylim([-3, 3])
ax_1_01.set_zlim([-3, 3])
ax_1_01.set_title("torch system")
ax_1_01.legend(loc="lower left")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Assignment and transformation of spatial data

A coordinate system stored in the `CoordinateSystemManager` can also get spatial data assigned to it. For example, this might be a sensor positions or geometry data in form of a point clouds. In this case it becomes the data's reference system and all values refer to its coordinate origin. 

Data can be assigned to a coordinate system with the `assign_data` function. It expects 3 parameters. The first one is the actual data. Currently, `xarray.DataArray` is the only supported type. The second parameter is a name of the data and the third one the name of the coordinate system that the data belongs to. In the following lines we define and assign some data to the `CoordinateSystemManager`.

In [8]:
sensor_positions_in_root = xr.DataArray(
    [[1, 0, 0], [2, 0, 0]], dims=["n", "c"], coords={"c": ["x", "y", "z"]}
)

specimen_geometry_in_specimen = xr.DataArray(
    [
        [0, 1, 0],
        [1, 1, 0],
        [2, 1, 0],
        [0, 2, 0],
        [1, 2, 0],
        [2, 2, 0],
        [0, 3, 0],
        [1, 3, 0],
        [2, 3, 0],
        [0, 4, 0],
        [1, 4, 0],
        [2, 4, 0],
    ],
    dims=["n", "c"],
    coords={"c": ["x", "y", "z"]},
)

csm.assign_data(sensor_positions_in_root,"sensors", "root")
csm.assign_data(specimen_geometry_in_specimen,"specimen geometry", "specimen")

With the `get_data` function, the unmodified data can be retrieved from the `CoordinateSystemManager` using its name as first parameter. 

In [10]:
assert specimen_geometry_in_specimen.identical(csm.get_data("specimen geometry"))

The optional second parameter can be used to get the data transformed into any of the `CoordinateSystemManager`s coordinate systems. You just need to specify the desired coordinate systems name:

In [12]:
specimen_geometry_in_root = csm.get_data("specimen geometry", "root")

sensor_positions_in_torch = csm.get_data("sensors", "torch")
specimen_geometry_in_torch = csm.get_data("specimen geometry", "torch")

Here is a plot of the specified data in the root and torch coordinate system.

In [13]:
lcs_root_in_root = tf.LocalCoordinateSystem()
lcs_torch_in_torch = tf.LocalCoordinateSystem()

# plot coordinate systems
fig_02 = plt.figure()
fig_02.set_size_inches(16, 8)
gs_02 = fig_02.add_gridspec(1, 2)
ax_0_02 = fig_02.add_subplot(gs_02[0, 0], projection="3d")
ax_1_02 = fig_02.add_subplot(gs_02[0, 1], projection="3d")

# first plot
vs.plot_coordinate_system(lcs_root_in_root, ax_0_02, color="r", label="root")
vs.plot_coordinate_system(lcs_flange_in_root, ax_0_02, color="g", label="flange")
vs.plot_coordinate_system(lcs_specimen_in_root, ax_0_02, color="y", label="specimen")
vs.plot_coordinate_system(lcs_torch_in_root, ax_0_02, color="b", label="torch")
ax_0_02.plot(specimen_geometry_in_root[:,0],specimen_geometry_in_root[:,1],specimen_geometry_in_root[:,2], "xc", label="specimen geometry")
ax_0_02.plot(sensor_positions_in_root[:,0],sensor_positions_in_root[:,1],sensor_positions_in_root[:,2], "xk", label="sensor positions")
    

ax_0_02.set_xlim([-3, 3])
ax_0_02.set_ylim([-3, 3])
ax_0_02.set_zlim([-3, 3])
ax_0_02.set_title("root system")
ax_0_02.legend(loc="lower left")  #


# second plot
vs.plot_coordinate_system(lcs_root_in_torch, ax_1_02, color="r", label="root")
vs.plot_coordinate_system(lcs_flange_in_torch, ax_1_02, color="g", label="flange")
vs.plot_coordinate_system(lcs_specimen_in_torch, ax_1_02, color="y", label="specimen")
vs.plot_coordinate_system(lcs_torch_in_torch, ax_1_02, color="b", label="torch")
ax_1_02.plot(specimen_geometry_in_torch[:,0],specimen_geometry_in_torch[:,1],specimen_geometry_in_torch[:,2], "xc", label="specimen geometry")
ax_1_02.plot(sensor_positions_in_torch[:,0],sensor_positions_in_torch[:,1],sensor_positions_in_torch[:,2], "xk", label="sensor positions")
ax_1_02.set_xlim([-3, 3])
ax_1_02.set_ylim([-3, 3])
ax_1_02.set_zlim([-3, 3])
ax_1_02.set_title("torch system")
ax_1_02.legend(loc="lower left")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

It is not necessary to attach data to a coordinate system to transform it to another one. The `CoordinateSystemManager` also provides the `transform_data` function for this purpose. It expects 3 parameters. The first parameter is the actual data and must be a `numpy.ndarray` or an `xarray.DataArray`. The second one is the source system name and the third one the target systems' name:

In [18]:
assert csm.transform_data(specimen_geometry_in_specimen,"specimen","root").identical(specimen_geometry_in_root)